![Podmanlogo](Pictures/podman-logo.png)

# Managing container images

In the previous section of this workshop we've seen how to run container images pulled from different registries but, what if I want to create a container image for my own application? No worries, podman has your back.

Podman can be used for building your own container images and push them to a container registry. In order to do it you need to first build a file called "Containerfile" and then used the "podman build" command to create your image out of the content of the Containerfile.

Lets see an example.

## Creating a new container image

Lets begin by creating a new Container file with the definition of our container image:

In [1]:

!echo -e "FROM docker.io/redhat/ubi9\nRUN dnf -y update && dnf -y install httpd  && dnf clean all\nCOPY index.html /var/www/html/index.html\nEXPOSE 80\nENTRYPOINT /usr/sbin/httpd -DFOREGROUND" > Containerfile
!cat Containerfile

FROM docker.io/redhat/ubi9
RUN dnf -y update && dnf -y install httpd  && dnf clean all
COPY index.html /var/www/html/index.html
EXPOSE 80
ENTRYPOINT /usr/sbin/httpd -DFOREGROUND


The output of the command is the content of our newly created Containerfile. Let's review it line by line.

First we have specified the "FROM" option, this line is mandatory, you need it to build your container image. With this line we specify the base container image that we will use for our application, in our example we've chosen a redhat/ubi9 image as base for our application. Choosing an image that you can trust is critical to maintain a secure environment, if your base image has vulnerabilities it may affect your application and potentially your whole enviroment. This is why we have chosen a Red Hat Universal Base Image (UBI) image for Red Hat Enterprise Linux 9 (RHEL9).

Red Hat creates UBI images that can be used securely, there are many options for your UBI images, from the standard RHEL 8 or RHEL 9 to a much secure micro image and even images for specific types of applications (like http, nodejs, etc). You can find a detailed list of all the CVE's and security status of Red Hat's UBI images in this website: https://catalog.redhat.com/software/containers/search
Also, if you're a Red Hat customer you will have full support for these images. But, if you're not, do not worry as some of this images are also uploaded to docker.io so you can access them and have enterprise grade security in your containers for free.

With that we now that our application will be running on top of a trusted and secured container image, now we need to add our software layer. To do so we added the second line in which we used the "RUN" option to specify the command that needs to be executed to install the reuqired software in our container image. If you don't have any experience with RHEL, we're just using the dnf command to update all packages, install the httpd and clean the cache (this last step is used only to have lighter container image).

Once the httpd package is installed we would like to customize the message that our containerized web server will show to the world. For that we used the "COPY" command. This command will copy a file from our local directory to inside the container that we are building, for our container we will be passing a file index.html from our local directory to the directory /var/www/html/ inside the container. We do this because we now that httpd uses that file to define the content to be posted through our web server.
If we want it to work we will need to create the index.html file so it can the be copied inside the container. Let's do it.

In [2]:
!echo "Hello from HPE Dev and Red Hat teams :)" > index.html
!cat index.html

Hello from HPE Dev and Red Hat teams :)


You see a message as output from the previous command, that is the message that our web server will be showing.

Next line in our Containerfile is using the "EXPOSE" instruction. The EXPOSE instruction informs Podman that the container listens on the specified network ports at runtime, in our case the port 80.

Last, an ENTRYPOINT allows you to configure the command that your container will run when started, in our case we want it to run the httpd binary to make sure our web server is running.

Next thing we need to do is building our newly created container image and that is quite a simple task. We just need to run the following command:

In [3]:
!podman build --tag rhel9:myhttpd -f ./Containerfile

STEP 1/5: FROM docker.io/redhat/ubi9
STEP 2/5: RUN dnf -y update && dnf -y install httpd  && dnf clean all
Updating Subscription Management repositories.
Unable to read consumer identity
Subscription Manager is operating in container mode.

This system is not registered with an entitlement server. You can use subscription-manager to register.

Red Hat Universal Base Image 9 (RPMs) - BaseOS  1.5 MB/s | 510 kB     00:00    
Red Hat Universal Base Image 9 (RPMs) - AppStre 4.3 MB/s | 1.7 MB     00:00    
Red Hat Universal Base Image 9 (RPMs) - CodeRea 688 kB/s | 187 kB     00:00    
Dependencies resolved.
 Package            Arch      Version                Repository            Size
Upgrading:
 curl-minimal       x86_64    7.76.1-23.el9_2.4      ubi-9-baseos-rpms    129 k
 libcurl-minimal    x86_64    7.76.1-23.el9_2.4      ubi-9-baseos-rpms    227 k
 libnghttp2         x86_64    1.43.0-5.el9_2.1       ubi-9-baseos-rpms     75 k

Transaction Summary
Upgrade  3 Packages

Total download siz

As you can see we passed two options to the podman build command. The first one, the tag option, specifies the name and tag of the container, in our case the name is rhel9 and myhttpd is the tag. Next we have used the "-f" option to specify the Containerfile that we want to use for building our container, by default podman build would take any file called Containerfile in the current directory but it's a best practice to use the "-f" option to make sure we use the correct file.

Our container image is now stored locally in our server and we can see it by running the following command:

In [9]:
!podman images

REPOSITORY                                                          TAG         IMAGE ID      CREATED         SIZE
localhost/rhel9                                                     myhttpd     2049b75e789a  33 minutes ago  242 MB
<none>                                                              <none>      95f0b9bd1c87  15 hours ago    314 MB
localhost/fedora                                                    myhttpd     bb83b1e58a8d  16 hours ago    353 MB
docker.io/redhat/ubi9-minimal                                       latest      d1a1af5ad0ce  44 hours ago    97.4 MB
quay.io/skupper/skupper-router                                      2.4.3       9f6d24d58351  10 days ago     275 MB
registry.fedoraproject.org/fedora                                   latest      a1cd3cbf8ada  2 weeks ago     195 MB
docker.io/redhat/ubi9                                               latest      07f698e0f2e0  2 weeks ago     217 MB
docker.io/library/nginx                                          

Our brand new container image is listed in the repository localhost/rhel9 and the tag myhttpd as we specified. Now our image is limited to be used in our local system, if we wanted to use it from other platforms we should upload the image to a container registry by running the command "podman push". We wont be doing this in this workshop, but you can look at all the details on how to use it by running "man podman-push".

As a last step let's run our image and check if it works fine.

In [6]:
!podman run -d -p 8080:80 --name myhttpd --rm rhel9:myhttpd

986fe5beb5fd53a864e812fa955974d73e9a94698e3e023abd1e0f0090cbff1b


We called our container "myhttpd" and pay attention to the option "-p". With this option we specified that the port 8080 of my system will be mapped to the port 80 of my container, if you remember correctly we specified in our Containerfile that our application was using the port 80 and now if anything within our network reaches the port 8080 of our RHEL machine it will be redirected to the port 80 of the container.

Lets test it by running a curl command against our local machine and the output should be the text we specified previously in our index.html file:

In [7]:
!curl localhost:8080

Hello from HPE Dev and Red Hat teams :)


As last step of this section we are going to clean up the environment. The command "podman rmi" is used to remove container images from the local system.

In [8]:
!podman stop --all
!podman rm --all
!podman rmi localhost/rhel9:myhttpd
!rm ./Containerfile
!rm index.html

986fe5beb5fd53a864e812fa955974d73e9a94698e3e023abd1e0f0090cbff1b
REPOSITORY  TAG         IMAGE ID    CREATED     SIZE
